In [1]:
import os
from glob import glob
from tqdm import tqdm
from pathlib import Path
import time

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from src.stats import calculate_p
from itertools import product

from deepjuice.model_zoo.options import get_model_options
model_info = get_model_options()

/home/emcmaho7/.conda/envs/deepjuice/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
figure_path = '../reports/figures/ResultsSummary'

In [3]:
rois = ['EVC', 'MT', 'EBA', 'LOC', 'pSTS', 'aSTS', 'FFA', 'PPA']
perturbations = ['shuffled', 'no nouns', 'only verbs', 'no verbs', 'only nouns',]
annotated_features = ['spatial expanse', 'agent distance', 'facingness',
                      'object directedness', 'joint action', 
                      'communication', 'valence', 'arousal']

def feature_naming(features): 
    feature_rename_mapping = {}
    for orig in features:
        revised = orig.replace('rating-', '').replace('_', ' ')
        if 'expanse' in orig: 
            revised = 'spatial expanse'
        elif 'object' in orig: 
            revised = 'object directedness'
        feature_rename_mapping[orig] = revised
    return feature_rename_mapping

In [4]:
def add_perturbation(file_, df_):
    if 'none' in file_:
        df_['perturbation'] = 'original'
    elif 'mask_nonverbs' in file_:
        df_['perturbation'] = 'only verbs'
    elif 'mask_nonnouns' in file_:
        df_['perturbation'] = 'only nouns'
    elif 'mask_nouns' in file_:
        df_['perturbation'] = 'no nouns'
    elif 'mask_verbs' in file_:
        df_['perturbation'] = 'no verbs'
    else:
        df_['perturbation'] = 'shuffled'
    return df_


def add_model_class(file_, df_):
    if 'Vision' in file_:
        df_['model_class'] = 'image'
    elif 'Video' in file_:
        df_['model_class'] = 'video'
    else:
        df_['model_class'] = 'language'
    return df_


def add_model_type(file_, df_):
    if 'Language' in file_:
        df_['model_type'] = 'language'
    else:
        df_['model_type'] = 'vision'
    return df_

In [5]:
behavior_reliability = pd.read_csv(f'../data/interim/ReorganizeBehavior/metadata.csv')
behavior_reliability.replace(feature_naming(behavior_reliability.feature.unique()), inplace=True)
behavior_reliability['feature'] = pd.Categorical(behavior_reliability['feature'], categories=annotated_features, ordered=True)
behavior_reliability

,feature,reliability
0,spatial expanse,0.719483
1,object directedness,0.928456
2,agent distance,0.885451
3,facingness,0.957481
4,joint action,0.768867
5,communication,0.762171
6,valence,0.747964
7,arousal,0.683388


# Perturbations

## Behavior

In [9]:
df = []
for file in glob('../data/interim/ModelAveraging/*LanguageBehaviorEncoding*pkl.gz'):
    df.append(add_perturbation(file, pd.read_pickle(file)))
df = pd.concat(df)
df['feature'] = df['feature'].replace(feature_naming(df.feature.unique()))
df.set_index(['perturbation', 'feature'], inplace=True)

stats = []
for perturbation in perturbations:
    for feature in annotated_features:
        null_diff = df.loc['original', feature]['r_null_dist'] - df.loc[perturbation, feature]['r_null_dist'] 
        diff = df.loc['original', feature]['test_score'] - df.loc[perturbation, feature]['test_score']
        p = calculate_p(null_diff, diff, n_perm_=len(null_diff), H0_='greater')
        stats.append({'perturbation': perturbation, 'feature': feature, 'p': p})
stats = pd.DataFrame(stats)
stats['perturbation'] = pd.Categorical(stats['perturbation'], categories=['original']+perturbations, ordered=True)
stats['feature'] = pd.Categorical(stats['feature'], categories=annotated_features, ordered=True)
stats = stats.sort_values(by=['perturbation', 'feature']).reset_index(drop=True)

In [10]:
run_summary = False
if run_summary: 
    for perturb in ['none', 'shuffle', 'mask_nouns', 'mask_verbs', 'mask_nonnouns', 'mask_nonverbs']:
        os.system(f'ml anaconda; conda activate deepjuice; python ../scripts/model_summary.py --model_class LanguageBehaviorEncoding --model_subpath {perturb}')

df = []
for file in glob('../data/interim/ModelSummary/*LanguageBehaviorEncoding*csv.gz'):
    df.append(add_perturbation(file, pd.read_csv(file)))
df = pd.concat(df)
df = df.loc[df.set == 'test'].reset_index(drop=True)
df['feature'] = df['feature'].replace(feature_naming(df.feature.unique()))
df['perturbation'] = pd.Categorical(df['perturbation'], categories=['original']+perturbations, ordered=True)
df['feature'] = pd.Categorical(df['feature'], categories=annotated_features, ordered=True)

In [11]:
def plot_perturbation_summary(df_, category, 
                              reliability_=None, out_file=None, stats=None,
                              y_var='test_score', y_label='Score ($r$)', 
                              palette_name=None):
    if palette_name is None:
        palette = sns.color_palette("husl", df_.perturbation.nunique())
    else:
        palette = sns.color_palette(palette_name, df_.perturbation.nunique())
    _, ax = plt.subplots(figsize=(8.5,4))
    color = ['gray', 'gray', 'gray']
    sns.stripplot(x=category, y=y_var, hue='perturbation', data=df_,
                  ax=ax, dodge=True, jitter=True, linewidth=.5,
                  zorder=1, palette=palette)
    sns.barplot(x=category, y=y_var, hue='perturbation', data=df_,
                dodge=True, saturation=.25, errorbar=None,
                legend=False, zorder=0, palette=palette)
    if reliability_ is not None:
        sns.pointplot(x=category, y='reliability', data=reliability_,
                      ax=ax, linestyle="none", errorbar=None, 
                      marker="_", markersize=30, markeredgewidth=7,
                      color='gray', alpha=.5, zorder=1)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ticks = ax.get_xticks()
    labels = [label.get_text() for label in ax.get_xticklabels()]
    ax.set_xticks(ticks)
    ax.set_xticklabels(labels, rotation=45, ha='right')
    ax.set_xlabel('')
    ax.set_ylabel(y_label)
    ax.legend(loc='center left', bbox_to_anchor=(1, 0.5),
              ncol=1, fancybox=True)
    plt.tight_layout()
    if out_file is not None:
        plt.savefig(out_file)

In [12]:
y_pos = {perturbation: (i_perturb+1)*-0.02 
         for i_perturb, perturbation in enumerate(perturbations)}
plot_perturbation_summary(df, 'feature', reliability_=behavior_reliability,
                          stats=stats,
                          y_var='score', stats_ypos=y_pos,
                          out_file='../reports/figures/ResultsSummary/behavior_pertubations.pdf')

TypeError: plot_perturbation_summary() got an unexpected keyword argument 'stats_ypos'

## Neural

In [ ]:
run_averaging = False
if run_averaging: 
    for perturb in ['none', 'shuffle', 'mask_nouns', 'mask_verbs', 'mask_nonnouns', 'mask_nonverbs']:
        os.system(f'ml anaconda; conda activate deepjuice; python ../scripts/model_averaging.py --model_class LanguageNeuralEncoding --model_subpath {perturb}')

df = []
for file in glob('../data/interim/ModelAveraging/*LanguageNeuralEncoding*pkl.gz'):
    df.append(add_perturbation(file, pd.read_pickle(file)))
df = pd.concat(df)

neural_metadata = pd.read_csv('../data/interim/ReorganziefMRI/metadata.csv')
df = df.merge(neural_metadata, on='voxel_id')

In [ ]:
def mean_of_arrays(series):
    # Stack arrays vertically and compute mean along the first axis (rows)
    return np.nanmean(np.vstack(series),axis=0)

stream_df = df.groupby('stream_name').agg({'test_score': 'mean', 
                                            'train_score': 'mean',
                                            'layer_relative_depth': 'mean',
                                            'reliability': 'mean',
                                            # 'r_null_dist': mean_of_arrays,
                                          }).reset_index()

In [ ]:
# stats = []
# for roi in rois:
#     for feature in annotated_features:
#         null_diff = df.loc['original', roi]['r_null_dist'] - df.loc[perturbation, roi]['r_null_dist'] 
#         diff = df.loc['original', roi]['test_score'] - df.loc[perturbation, roi]['test_score']
#         p = calculate_p(null_diff, diff, n_perm_=len(null_diff), H0_='greater')
#         stats.append({'perturbation': perturbation, 'roi_name': roi, 'p': p})
# stats = pd.DataFrame(stats)
# stats['perturbation'] = pd.Categorical(stats['perturbation'], categories=['original']+perturbations, ordered=True)
# stats['roi_name'] = pd.Categorical(stats['roi_name'], categories=rois, ordered=True)
# stats.sort_values(by=['roi_name', 'perturbation'], inplace=True)
# stats.set_index(['roi'], inplace=True)

# stats['sig_text'] = 'none'
# stats.loc[(0.05 > stats.p) & (stats.p >= 0.01), 'sig_text'] = '*'
# stats.loc[(0.01 > stats.p) & (stats.p >= 0.001), 'sig_text'] = '**'
# stats.loc[(0.001 > stats.p), 'sig_text'] = '***'

In [ ]:
run_summary = False
if run_summary: 
    for perturb in ['none', 'shuffle', 'mask_nouns', 'mask_verbs', 'mask_nonnouns', 'mask_nonverbs']:
        os.system(f'ml anaconda; conda activate deepjuice; python ../scripts/model_summary.py --model_class LanguageNeuralEncoding --model_subpath {perturb}')

df = []
for file in glob('../data/interim/ModelSummary/*LanguageNeuralEncoding*csv.gz'):
    df.append(add_perturbation(file, pd.read_csv(file)))
df = pd.concat(df)
df = df.loc[df.set == 'test'].reset_index(drop=True)
df['perturbation'] = pd.Categorical(df['perturbation'], categories=['original']+perturbations, ordered=True)
df['roi_name'] = pd.Categorical(df['roi_name'], categories=rois, ordered=True)

In [ ]:
def plot_neural_perturbation_summary(df_, category, 
                                      reliability_=None, out_file=None, stats=None,
                                      y_var='test_score', y_label='Score ($r$)', 
                                      palette_name=None, stats_ypos=None):
    if palette_name is None:
        palette = sns.color_palette("Paired", df_.perturbation.nunique())
    else:
        palette = sns.color_palette(palette_name, df_.perturbation.nunique())
    _, ax = plt.subplots(figsize=(8.5,4))
    color = ['gray', 'gray', 'gray']
    sns.stripplot(x=category, y=y_var, hue='perturbation', data=df_,
                  ax=ax, dodge=True, jitter=True, linewidth=.5,
                  zorder=1, palette=palette)
    bars = sns.barplot(x=category, y=y_var, hue='perturbation', data=df_,
                        dodge=True, errorbar=None, saturation=.25,
                        legend=False, zorder=0, palette=palette)
    
    if stats is not None: 
        # Adding significance lines    
        original_container = bars.containers[0]
        containers = bars.containers[1:]
        
        counter = -1
        for i_container, container in enumerate(containers):
            for i_bar, bar in enumerate(container.patches):
                counter += 1
                if stats.iloc[counter]['p'] < 0.05: 
                    x_min = original_container.patches[i_bar].get_x() + bar.get_width() / 2
                    x_max = bar.get_x() + bar.get_width() / 2
                    yval = stats_ypos[stats.iloc[counter]['perturbation']]
                    plt.hlines(y=yval, xmin=x_min, xmax=x_max,
                               color=bar.get_facecolor(), linewidth=2)
                    
    if reliability_ is not None:
        sns.pointplot(x=category, y='reliability', data=reliability_,
                      ax=ax, linestyle="none", errorbar=None, 
                      marker="_", markersize=30, markeredgewidth=5,
                      color='gray', alpha=.5, zorder=1)

    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ticks = ax.get_xticks()
    labels = [label.get_text() for label in ax.get_xticklabels()]
    ax.set_xticks(ticks)
    ax.set_xlabel('')
    ax.set_ylabel(y_label)
    ax.legend(loc='center right', bbox_to_anchor=(1, 0.5),
              ncol=1, fancybox=True)
    if out_file is not None:
        plt.tight_layout()
        plt.savefig(out_file)

In [ ]:
plot_neural_perturbation_summary(df, 'roi_name', 
                          y_var='score',
                          out_file=f'{figure_path}/neural_pertubations.pdf')

# Image vs Video vs Language

## Behavior

In [ ]:
def plot_combined_behavior_summary(df_, reliability_, hue='model_class', out_file=None):
    anchor = (.7, 1.2)
    if hue == 'model_class':
        sns.set_palette(['tab:cyan', 'tab:purple'])
        anchor = (.7, 1.2)
    elif hue == 'model_type':
        sns.set_palette(['tab:cyan', 'tab:purple'])
        anchor = (.65, 1.2)
    elif hue == 'architecture_type':
        sns.set_palette(['tab:orange', 'tab:green'])
    elif hue == 'task_cluster':
        sns.set_palette(['tab:red', 'gold'])
    elif hue == 'modality':
        sns.set_palette(['tab:pink', 'skyblue'])
    _, ax = plt.subplots(figsize=(8.5,4.25))
    
    if df_[hue].nunique() == 3:
        dodge = 0.55
        markersize = 14
    else:
        dodge = 0.4
        markersize = 18

    sns.stripplot(x='feature', y='score', hue=hue, data=df_,
                  ax=ax, dodge=True, jitter=True, linewidth=.5, alpha=0.25, zorder=0, legend=False)
    sns.pointplot(x='feature', y='score', hue=hue, data=df_,
                  dodge=dodge, linestyle="none", errorbar=None,
                  marker="_", markersize=markersize, markeredgewidth=3, zorder=1)
    sns.pointplot(x='feature', y='reliability', data=reliability_,
                  ax=ax, linestyle="none", errorbar=None, 
                  marker="_", markersize=30, markeredgewidth=5,
                  color='gray', alpha=.5, zorder=1)
    
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ticks = ax.get_xticks()
    labels = [label.get_text() for label in ax.get_xticklabels()]
    ax.set_xticks(ticks)
    ax.set_xticklabels(labels, rotation=45, ha='right')
    ax.set_xlabel('Behavioral ratings')
    ax.set_ylabel('Score ($r$)')
    ax.legend(loc='upper right', bbox_to_anchor=anchor,
              ncol=df_[hue].nunique(), fancybox=True)
    plt.tight_layout()
    if out_file is not None:
        plt.savefig(out_file)

In [ ]:
top = '../data/interim/ModelSummary/'
files = [f'{top}/LanguageBehaviorEncoding_none.csv.gz']
files += [f'{top}/VisionBehaviorEncoding_grouped_average.csv.gz']
files += [f'{top}/VideoBehaviorEncoding_None.csv.gz']
df = []
for file in files:
    cdf = pd.read_csv(file)
    add_model_class(file, cdf)
    add_model_type(file, cdf)
    df.append(cdf)
df = pd.concat(df)
df.replace(feature_naming(df.feature.unique()), inplace=True)
df['model_class'] = pd.Categorical(df['model_class'], ordered=True, 
                                   categories=['image', 'video', 'language'])
df['model_type'] = pd.Categorical(df['model_type'], ordered=True, 
                                   categories=['vision', 'language'])
df['feature'] = pd.Categorical(df['feature'], ordered=True, 
                                   categories=annotated_features)
df = df.loc[df.set == 'test'].reset_index(drop=True)

In [ ]:
plot_combined_behavior_summary(df, behavior_reliability,
                               out_file=f'{figure_path}/behavior_summary.pdf')

In [ ]:
plot_combined_behavior_summary(df, behavior_reliability, hue='model_type',
                               out_file=f'{figure_path}/behavior_vision-language.pdf')

In [ ]:
image_df = df.loc[df.model_class == 'image'].reset_index(drop=True).merge(model_info, on='model_uid')

In [ ]:
conv_meta_max = image_df[image_df['architecture_type'].isin(['Convolutional', 'Transformer'])]
mapping = {i: i.lower() for i in ['Convolutional', 'Transformer']}
conv_meta_max.replace(mapping, inplace=True)
conv_meta_max['architecture_type'] = pd.Categorical(conv_meta_max['architecture_type'], 
                                                    categories=mapping.values(), ordered=True)
plot_combined_behavior_summary(conv_meta_max, behavior_reliability,
                               hue='architecture_type',
                               out_file=f'{figure_path}/behavior_conv-transformer.pdf')

In [ ]:
categories = ['Supervised', 'Self-Supervised']

resnet_info = model_info[(model_info['model_source'].isin(['torchvision', 'vissl', 'dino', 'timm'])) & (model_info['architecture'].str.contains('resnet50'))]
resnet_df = image_df[image_df['model_uid'].isin(resnet_info.model_uid)]
sup_meta_max = resnet_df[resnet_df['task_cluster'].isin(categories)]
mapping = {i: i.lower() for i in categories}
sup_meta_max.replace(mapping, inplace=True)
sup_meta_max['task_cluster'] = pd.Categorical(sup_meta_max['task_cluster'], 
                                                    categories=mapping.values(), ordered=True)

plot_combined_behavior_summary(sup_meta_max, behavior_reliability,
                               hue='task_cluster',
                               out_file=f'{figure_path}/behavior_sup-selfsup.pdf')

In [ ]:
categories = ['Vision', 'Vision-Language']
vis_lang_meta_max = image_df.loc[(image_df['task_cluster']=='Supervised') & (image_df['modality']==categories[0]) | (image_df['modality']==categories[1])]
mapping = {i: i.lower() for i in categories}
vis_lang_meta_max.replace(mapping, inplace=True)
vis_lang_meta_max['modality'] = pd.Categorical(vis_lang_meta_max['modality'], 
                                            categories=mapping.values(), ordered=True)

plot_combined_behavior_summary(vis_lang_meta_max, behavior_reliability, hue='modality',
                               out_file=f'{figure_path}/behavior_vision-vislanguage.pdf')

## Neural

In [ ]:
top = '../data/interim/ModelSummary/'
files = [f'{top}/LanguageNeuralEncoding_none.csv.gz']
files += [f'{top}/VisionNeuralEncoding_grouped_average.csv.gz']
files += [f'{top}/VideoNeuralEncoding_None.csv.gz']
df = []
for file in files:
    cdf = pd.read_csv(file)
    add_model_class(file, cdf)
    add_model_type(file, cdf)
    df.append(cdf)
df = pd.concat(df)
df['model_class'] = pd.Categorical(df['model_class'], ordered=True, 
                                   categories=['image', 'video', 'language'])
df['model_type'] = pd.Categorical(df['model_type'], ordered=True, 
                                   categories=['vision', 'language'])
df = df.loc[df.set == 'test'].reset_index(drop=True)

In [ ]:
def plot_combined_neural(df_, hue='model_class', out_file=None):
    anchor = (1, 1.2)
    if hue == 'model_class':
        sns.set_palette(['tab:cyan', 'tab:purple'])
        anchor = (1, 1.2)
    elif hue == 'model_type':
        sns.set_palette(['tab:cyan', 'tab:purple'])
    elif hue == 'architecture_type':
        sns.set_palette(['tab:orange', 'tab:green'])
    elif hue == 'task_cluster':
        sns.set_palette(['tab:red', 'gold'])
    elif hue == 'modality':
        sns.set_palette(['tab:pink', 'skyblue'])
    regions = ['Lateral ROIs', 'Ventral ROIs']
    
    if df_[hue].nunique() == 3:
        dodge = 0.55
        markersize = 14
    else:
        dodge = 0.4
        markersize = 18

    _, axes = plt.subplots(1, 2, figsize=(6.5, 3.5), gridspec_kw={'width_ratios': [3, 1]},
                          sharey=True)
    for i_ax, (ax, cur_rois) in enumerate(zip(axes,
                                              [['EVC', 'MT', 'EBA', 'LOC', 'pSTS', 'aSTS'],
                                               ['FFA', 'PPA']])):
        legend = True if i_ax == 0 else False
        roi_positions = {roi: i for i, roi in enumerate(cur_rois)}

        cur_scores = df_[df_.roi_name.isin(cur_rois)]
        cur_scores['roi_name'] = pd.Categorical(cur_scores['roi_name'],
                                                categories=cur_rois, ordered=True)
        sns.stripplot(x='roi_name', y='score', hue=hue, data=cur_scores,
                      ax=ax, dodge=True, jitter=True, linewidth=.5,
                      alpha=0.25, zorder=0, legend=False)
        sns.pointplot(x='roi_name', y='score', hue=hue,
                      data=cur_scores, legend=legend,
                      ax=ax, dodge=dodge, linestyle="none", errorbar=None,
                      marker="_", markersize=markersize,
                      markeredgewidth=3, zorder=1)
        sns.pointplot(x='roi_name', y='reliability', data=cur_scores,
              linestyle="none", errorbar=None, color='gray',
              marker="_", markersize=45, markeredgewidth=5,
              alpha=.5, zorder=1, ax=ax)

        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)
        ax.set_xlabel(regions[i_ax])
        if i_ax == 0: 
            ax.set_ylabel('Score ($r$)')
            ax.legend(loc='upper right', bbox_to_anchor=anchor,
                      ncol=df_[hue].nunique(), fancybox=True)
    plt.tight_layout()
    if out_file is not None:
        plt.savefig(out_file)

In [ ]:
plot_combined_neural(df, hue='model_class',
                     out_file=f'{figure_path}/neural_summary.pdf')

In [ ]:
plot_combined_neural(df, hue='model_type',
                     out_file=f'{figure_path}/neural_vision-language.pdf')

In [ ]:
image_df = df.loc[df.model_class == 'image'].reset_index(drop=True).merge(model_info, on='model_uid')

In [ ]:
categories = ['Convolutional', 'Transformer']
hue = 'architecture_type' 

conv_meta_max = image_df[image_df[hue].isin(categories)]
mapping = {i: i.lower() for i in categories}
conv_meta_max.replace(mapping, inplace=True)
conv_meta_max[hue] = pd.Categorical(conv_meta_max[hue],
                                    categories=mapping.values(), ordered=True)
plot_combined_neural(conv_meta_max, hue=hue,
                     out_file=f'{figure_path}/neural_conv-transformer.pdf')

In [ ]:
categories = ['Supervised', 'Self-Supervised']
hue = 'task_cluster' 

resnet_info = model_info[(model_info['model_source'].isin(['torchvision', 'vissl', 'dino', 'timm'])) & (model_info['architecture'].str.contains('resnet50'))]
resnet_df = image_df[image_df['model_uid'].isin(resnet_info.model_uid)]
sup_meta_max = resnet_df[resnet_df[hue].isin(categories)]
mapping = {i: i.lower() for i in categories}
sup_meta_max.replace(mapping, inplace=True)
sup_meta_max[hue] = pd.Categorical(sup_meta_max[hue],
                                   categories=mapping.values(), ordered=True)

plot_combined_neural(sup_meta_max, hue=hue,
                     out_file=f'{figure_path}/neural_sup-selfsup.pdf')

In [ ]:
categories = ['Vision', 'Vision-Language']
hue = 'modality' 

vis_lang_meta_max = image_df.loc[(image_df['task_cluster']=='Supervised') & (image_df[hue]==categories[0]) | (image_df[hue]==categories[1])]
mapping = {i: i.lower() for i in categories}
vis_lang_meta_max.replace(mapping, inplace=True)
vis_lang_meta_max[hue] = pd.Categorical(vis_lang_meta_max[hue], 
                                            categories=mapping.values(), ordered=True)

plot_combined_neural(vis_lang_meta_max, hue=hue,
                     out_file=f'{figure_path}/neural_vision-vislanguage.pdf')